In [1]:
import time
import casadi
import numpy as np
from utils.RobotConfig import RobotConfig
from utils.Lagrange import *
from utils.LGL_collocation import LGL_collocation
from utils.LocalChart import *
import pinocchio as pin
from pinocchio import casadi as cpin
import matplotlib.pyplot as plt
import scipy

In [2]:
Ntp = 20     # Number of collocation points / basis (degree: Ntp-1)

### Robot Model

In [3]:
config_file = 'config/SimpleConfig.yaml'
q_waypoints = np.array([[0, np.pi/4, 0, 0, 0, 0], 
                    [-np.pi/6, np.pi/12, np.pi/6, -np.pi/6, -np.pi/3, -np.pi/3],
                    [-np.pi/4, np.pi/4, np.pi/3, 0, -np.pi/6, -np.pi/2]])

Robot = RobotConfig(config_file)
Robot.homePosition = q_waypoints[0]
Robot.lowerVelocityLimit = np.pi/2 * np.array([-1, -1, -1, -1, -1, -1])  # rad/s
Robot.upperVelocityLimit = np.pi/2 * np.array([1, 1, 1, 1, 1, 1])
Robot.lowerTorqueLimit = 100 * np.array([-1, -1, -1, -1, -1, -1]) 
Robot.upperTorqueLimit = 100 * np.array([1, 1, 1, 1, 1, 1])


ngc = Robot.model.nq   # Number of joints
ndof = Robot.model.nv   # Number of DOF
nx = 2*ngc    # Number of states
cx = casadi.SX.sym("x", nx, 1)
ctau = casadi.SX.sym("tau", ndof, 1)
cacc = cpin.aba(Robot.cmodel, Robot.cdata, cx[:ngc], cx[ngc:], ctau) # Forward dynamics


q0 = q_waypoints[0]
qT = q_waypoints[-1]
tau0 = pin.rnea(Robot.model, Robot.data, q0, np.zeros(ndof), np.zeros(ndof))
tauT = pin.rnea(Robot.model, Robot.data, qT, np.zeros(ndof), np.zeros(ndof))
print(tau0)
print(tauT)

[  0.         -36.08572402  -9.26563407   0.97299901  -0.09997149
   0.        ]
[ 8.88178420e-16 -2.17616448e+01  5.05844512e+00  1.31084531e+00
 -1.18267539e-01  0.00000000e+00]


### Gauss–Legendre Collocation

In [4]:
Np = q_waypoints.shape[0]-1  # Number of phase (dersired waypoints: N+1)
T = Np*Ntp  # Number of total collocation points / optimization variables

LGL_Pts, LGL_Wts = Normalize_LGL_Pts_Wts(Ntp)
DL,_ = differentiation_matrix(Ntp, LGL_Pts)

LGL = LGL_collocation(Ntp, ngc, ndof)
StatesDiff, StateDiff, LocalsDiff, LocalDiff= LGL.getDiffFuncs()

Feq = casadi.Function(
    "Dyn",
    [cx, ctau],
    [casadi.vertcat(cx[ngc:], cacc)],
)

### Nonlinear Programming Problem (NLP)

In [5]:
opti = casadi.Opti()
# Optimization variables
var_xs = [opti.variable(nx) for _ in range(T)]
var_us = [opti.variable(ndof) for _ in range(T)]
var_deltaT = [opti.variable(1) for _ in range(Np)]

In [6]:
# Constraints
for i in range(Np):
    opti.subject_to(var_deltaT[i] >= 0.0)

    for tp in range(Ntp):
        t = i*Ntp + tp
        # dynamics
        opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[tp,:]) 
                        == Feq(var_xs[t], var_us[t]))
        
        # joint constraints
        opti.subject_to(var_xs[t][:ngc] <= Robot.upperPositionLimit)
        opti.subject_to(var_xs[t][:ngc] >= Robot.lowerPositionLimit)
        # joint velocity constraints
        opti.subject_to(var_xs[t][ngc:] <= Robot.upperVelocityLimit)
        opti.subject_to(var_xs[t][ngc:] >= Robot.lowerVelocityLimit)
        # torque constraints
        opti.subject_to(var_us[t] <= Robot.upperTorqueLimit)
        opti.subject_to(var_us[t] >= Robot.lowerTorqueLimit)

# waypoints constraints (N-1 middle waypoints)
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp][:ngc] == q_waypoints[i])

# continuity
for i in range(1, Np):
    opti.subject_to(var_xs[i*Ntp-1] == var_xs[i*Ntp])
    opti.subject_to(var_us[i*Ntp-1] == var_us[i*Ntp])

# continuity of derivatives
for i in range(Np-1):
    opti.subject_to(StatesDiff(casadi.vertcat(*var_xs[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:]) 
                    == StatesDiff(casadi.vertcat(*var_xs[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i+1], DL[0,:]))
    
    opti.subject_to(LocalDiff(casadi.vertcat(*var_us[i*Ntp: (i+1)*Ntp]), var_deltaT[i], DL[Ntp-1,:])
                   == LocalDiff(casadi.vertcat(*var_us[(i+1)*Ntp: (i+2)*Ntp]), var_deltaT[i], DL[0,:]))

# boundary conditions
opti.subject_to(var_xs[0][:ngc] == q0)
opti.subject_to(var_xs[-1][:ngc] == qT)
opti.subject_to(var_xs[0][ngc:] == 0)
opti.subject_to(var_xs[-1][ngc:] == 0)
#opti.subject_to(var_us[0] == tau0)
#opti.subject_to(var_us[-1] == tauT)

# Objective
totalcost = 0

R = np.eye(ndof)
for i in range(Np):
    for tp in range(Ntp):
        var_xs_Phase = var_xs[i*Ntp: (i+1)*Ntp]
        dq = [dq_Phase[ngc:] for dq_Phase in var_xs_Phase]
        ddq = StateDiff(casadi.vertcat(*dq), var_deltaT[i], DL[tp,:])
        totalcost += var_deltaT[i]/2 * LGL_Wts[tp] * casadi.mtimes([ddq.T, R, ddq])
        
    totalcost += var_deltaT[i]

opti.minimize(totalcost)
opti.solver("ipopt") 

for x in var_xs:
    opti.set_initial(x, np.concatenate([q0, np.zeros(ngc)]))
for u in var_us:
    opti.set_initial(u, tau0)
for t in var_deltaT:
    opti.set_initial(t, 1)


In [7]:
try:
    sol = opti.solve_limited()
    sol_xs = [opti.value(var_x) for var_x in var_xs]
    sol_us = [opti.value(var_u) for var_u in var_us]
    sol_T = [opti.value(var_t) for var_t in var_deltaT]
except:
    print("ERROR in convergence, plotting debug info.")
    sol_xs = [opti.debug.value(var_x) for var_x in var_xs]
    sol_us = [opti.debug.value(var_u) for var_u in var_us]


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.16, running with linear solver MUMPS 5.7.2.

Number of nonzeros in equality constraint Jacobian...:    15216
Number of nonzeros in inequality constraint Jacobian.:     1442
Number of nonzeros in Lagrangian Hessian.............:    18962

Total number of variables............................:      722
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      546
Total number of inequality c

In [8]:
print(sol_T)
print(f"Total time: {sum(sol_T)}")

T_List = [0] + [sum(sol_T[:i+1]) for i in range(Np)]



[3.0003991033774398, 2.3745626414482217]
Total time: 5.3749617448256615


In [11]:
from utils.ObstacleSpheres import ObsConfig

obs = ObsConfig(config_file)
obs.spheres_from_robot(q0 = q_waypoints[0].astype(float).tolist())  # Get initial robot obstacle spheres
obs.spheres_from_env()

cq = casadi.SX.sym('q', Robot.ndof)
get_obstacle_distances = casadi.Function(
    'get_obstacle_distances', 
    [cq], 
    [casadi.vertcat(*Robot.obstacle_distances(cq, obs.env_spheres, obs.robot_spheres))]
    )

for t in range(T):
    print(f"Collision Distance {t}: {get_obstacle_distances(sol_xs[t][:ngc])}, \n State:{Robot.collision_status(sol_xs[t][:ngc], obs.env_spheres, obs.robot_spheres)}")

Collision Distance 0: [0.453906, 0.416406, 0.0872761, 0.22627, 0.458041, 0.426905, 0.311941, 0.333838, 0.176466, 0.259174, 0.109124, 0.231855, 0.230739, 0.288133, 0.411669, 0.40316, 0.478139, 0.43965, 0.0600583, 0.208527, 0.0192186, 0.326221, 0.0199047, 0.214053, 0.00449313, 0.235411, 0.00495637, 0.279432, 0.0401787, 0.209295, 0.00189849, 0.257565, 0.022891, 0.333429, 0.0101919, 0.222583, 0.0379068, 0.347115, 0.029066, 0.338275, 0.0478279, 0.376977, 0.0379479, 0.339661, 0.0247187, 0.264136, 0.0294815, 0.268899, 0.0291062, 0.292848, 0.0198831, 0.258169, 0.0515788, 0.290572], 
 State:False
Collision Distance 1: [0.453906, 0.416406, 0.0872818, 0.226334, 0.458032, 0.426907, 0.311937, 0.333859, 0.176467, 0.259218, 0.109129, 0.231912, 0.230738, 0.288167, 0.411663, 0.40317, 0.478129, 0.439649, 0.0600728, 0.208594, 0.0192427, 0.326323, 0.0199231, 0.21413, 0.00451363, 0.235496, 0.00497903, 0.279527, 0.040196, 0.209366, 0.00192027, 0.257655, 0.0229173, 0.333532, 0.0102112, 0.222664, 0.0379233, 0

### Inverse dynamics


In [ ]:
nsample = 100
fig, axs = plt.subplots(nv,1)
for i in range(Np):
    sample_t = np.linspace(T_List[i], T_List[i+1], nsample)
    collo_pts = sol_T[i]/2 * (LGL_Pts+1) + T_List[i]

    t = casadi.SX.sym("t")
    inter_xs = casadi.Function(
        "xt", 
        [t], [LagrangeInterpolation(t, collo_pts, sol_xs[i*Ntp: (i+1)*Ntp])]
    )
    inter_us = casadi.Function(
        "ut", 
        [t], [LagrangeInterpolation(t, collo_pts, sol_us[i*Ntp: (i+1)*Ntp])]
    )
    inter_dxs = casadi.Function(
        "dxt", 
        [t], [DiffLangrangeInterpolation(t, collo_pts, sol_xs[i*Ntp: (i+1)*Ntp])]
    )


    sample_q = np.array([inter_xs(t)[:nq] for t in sample_t])
    sample_dq = np.array([inter_xs(t)[nq:] for t in sample_t])
    sample_ddq = np.array([inter_dxs(t)[nv:] for t in sample_t])

    # from interpolation
    sample_u = np.array([inter_us(t) for t in sample_t])
    
    # from dynamics equations
    tau_id = np.array([pin.rnea(model, data, q, dq, ddq) for q, dq, ddq in zip(sample_q, sample_dq, sample_ddq)])
    # plot sample_u and tau
    
    for j in range(nv): 
        axs[j].plot(sample_t, sample_u[:,j], label=f'u{j}',color='blue')
        axs[j].plot(sample_t, tau_id[:,j], label=f'tau{j}',color='orange',linestyle='dashed')

plt.show()

### Forward Dynamics

In [ ]:
def ForwardDynamics(x, t, inter_us):
    q = x[:nq]
    dq = x[nq:]
    u = np.squeeze(inter_us(t))
    ddq = pin.aba(model, data, q, dq, u)
    return np.concatenate([dq, ddq])

q_forward = [None]*Np
dq_forward = [None]*Np
ddq_forward = [None]*Np

x0 = sol_xs[0]   # initial condition
for i in range(Np):
    time_grid = np.linspace(T_List[i], T_List[i+1], nsample)
    collo_pts = sol_T[i]/2 * (LGL_Pts+1) + T_List[i]
    t = casadi.SX.sym("t")
    inter_us = casadi.Function(
        "ut", 
        [t], [LagrangeInterpolation(t, collo_pts, sol_us[i*Ntp: (i+1)*Ntp])]
    )
    x_forward = scipy.integrate.odeint(ForwardDynamics, x0, time_grid, args = (inter_us,))
    x0 = x_forward[-1]
    
    q_forward[i] = x_forward[:,:nq]
    dq_forward[i] = x_forward[:,nq:]
    u_grid = np.array([np.squeeze(inter_us(t)) for t in time_grid])
    ddq_forward[i] = np.array([pin.aba(model, data, q, dq, u) for q, dq, u in zip(x_forward[:,:nq], x_forward[:,nq:], u_grid)])


fig1, axs1 = plt.subplots(nq, 1)  # joint positions
fig2, axs2 = plt.subplots(nq, 1)  # joint velocities
fig3, axs3 = plt.subplots(nq, 1)  # joint accelerations

for i in range(Np):
    time_grid = np.linspace(T_List[i], T_List[i+1], nsample)
    collo_pts = sol_T[i]/2 * (LGL_Pts+1) + T_List[i]

    t = casadi.SX.sym("t")
    inter_xs = casadi.Function(
        "xt", 
        [t], [LagrangeInterpolation(t, collo_pts, sol_xs[i*Ntp: (i+1)*Ntp])]
    )
    inter_dxs = casadi.Function(
        "dxt", 
        [t], [DiffLangrangeInterpolation(t, collo_pts, sol_xs[i*Ntp: (i+1)*Ntp])]
    )

    # Interpolated trajectory / Nominal trajectory
    sample_q = np.array([np.squeeze(inter_xs(t)[:nq]) for t in time_grid])
    sample_dq = np.array([np.squeeze(inter_xs(t)[nq:])  for t in time_grid])
    sample_ddq = np.array([np.squeeze(inter_dxs(t)[nv:]) for t in time_grid])


    for j in range(nq):
        print(q_forward[i][:,j])
        print(sample_q[:,j])
        axs1[j].plot(time_grid, q_forward[i][:,j]-sample_q[:,j], label=f'Integrate_q{j}',color='blue')

    for j in range(nq):
        axs2[j].plot(time_grid, dq_forward[i][:,j], label=f'Integrate_dq{j}',color='blue')
        axs2[j].plot(time_grid, sample_dq[:,j], label=f'Nominal_dq{j}',color='orange',linestyle='dashed')

    for j in range(nv):
        axs3[j].plot(time_grid, ddq_forward[i][:,j], label=f'Integrate_ddq{j}',color='blue')   # blue
        axs3[j].plot(time_grid, sample_ddq[:,j], label=f'Nominal_ddq{j}', color='orange',linestyle='dashed')   
